In [ ]:
"""
Agenda

Part 1: Chat with PDF
User uploads a PDF.

Chunk it, embed it using Hugging Face embeddings.

Store the vectors using FAISS.

Use a retriever + LLM to answer user questions from the PDF.


Part 2: Zero-Shot ReAct Agent with Tools
Agent reasons and selects tools (Wikipedia, Python REPL, etc.).

Ask multi-hop questions like: “What’s the square root of the population of France?”

Agent handles it without needing external docs.

"""

In [ ]:
# --- INSTALLATION (fast) ---
!pip install -q langchain openai faiss-cpu tiktoken pypdf sentence-transformers wikipedia langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
# --- SETUP ---
import os
os.environ["OPENAI_API_KEY"] = "1234 fake key"

In [ ]:
# --- PART 1: PDF QA SYSTEM ---
# Load PDF, chunk, embed, and run QA

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from google.colab import files

In [ ]:
# Upload PDF
uploaded = files.upload()
pdf_path = next(iter(uploaded))



Saving CreatingLargeLanguageModelApplicationsUtilizingLangChain-APrimeronDevelopingLLMAppsFast.pdf to CreatingLargeLanguageModelApplicationsUtilizingLangChain-APrimeronDevelopingLLMAppsFast.pdf


In [ ]:
# Load and split PDF
loader = PyPDFLoader(pdf_path)
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(pages)

In [ ]:
# Create embeddings and vector store
embeddings = HuggingFaceEmbeddings()
vectordb = FAISS.from_documents(docs, embeddings)

<ipython-input-8-20bc72226928>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-8-20bc72226928>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingf

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# RetrievalQA chain using OpenAI
retriever = vectordb.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), retriever=retriever)

<ipython-input-9-14f1fbddd24b>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), retriever=retriever)


In [ ]:
# Ask a question based on the PDF content
print("PDF QA Example:")
pdf_question = "Summarize the key points from the document."
pdf_answer = qa_chain.run(pdf_question)
print(pdf_answer)

PDF QA Example:


<ipython-input-10-46ac64633c0b>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  pdf_answer = qa_chain.run(pdf_question)


 The document discusses the use of LLMs to answer questions about the content of various documents, such as PDFs, webpages, and CSV files. It also mentions the steps involved in this process and the potential benefits for auditing purposes and system performance. Additionally, it mentions the importance of historical knowledge and evidence in answering questions and making judgments.


In [ ]:
from IPython.display import display, Markdown

# Title
display(Markdown("### 📄 PDF QA Example"))

# Display the question in a friendly format
pdf_question = "Summarize the key points from the document."
display(Markdown(f"**Question:** {pdf_question}"))

# Run the QA chain
pdf_answer = qa_chain.run(pdf_question)

# Display the answer nicely
display(Markdown(f"**Answer:**\n\n{pdf_answer}"))

### 📄 PDF QA Example

**Question:** Summarize the key points from the document.

**Answer:**

 The use of LLMs for question answering in various applications, such as auditing and system performance, can be beneficial. One common and complex application is using LLMs to answer questions about a document's content. Steps involved in this process are depicted in Figure 4. It is important to respect historical evidence and use it to support explanations and judgments. Additionally, LLMs can be trained to process data for various use cases. This task involves extracting information such as whether an item was purchased as a gift, how many days it took for the product to arrive, and any sentences about the value or price. A follow-up response should include a summary of these key points.

In [ ]:
# Define your application-oriented questions
questions = [
    "Explain how LangChain can be used to build a document-based Q&A system. What components are involved?",
    "What are the advantages and trade-offs between the stuff, map-reduce, refine, and map-rerank methods in LangChain?",
    "How does LangChain support the use of agents and tools for dynamic task execution?"
]

# Loop through and ask each question, then format the output nicely
for i, question in enumerate(questions, 1):
    display(Markdown(f"### Question {i}"))
    display(Markdown(f"**{question}**"))

    answer = qa_chain.run(question)  # using RetrievalQA chain already defined
    display(Markdown(f"**Answer:**\n\n{answer}\n"))

### Question 1

**Explain how LangChain can be used to build a document-based Q&A system. What components are involved?**

**Answer:**

 LangChain can be used to build a document-based question answering system by utilizing its main components such as prompts, memory, chains, and agents. The "prompt" component provides the input to the system, which includes the user's question, a set of few shot examples, and instructions for the system to process the input. The "memory" component allows the system to store and retrieve information from a database. The "chains" component combines an LLM with a prompt to carry out a sequence of operations on the input text or data. Finally, the "agents" component provides a way to execute queries and inspect tables in the LangChain database. By utilizing these components, LangChain can be used to build a robust and efficient document-based Q&A system.


### Question 2

**What are the advantages and trade-offs between the stuff, map-reduce, refine, and map-rerank methods in LangChain?**

**Answer:**


The advantages of using the stuff method in LangChain include the ability to iteratively build upon previous documents, potentially leading to longer and more comprehensive answers. However, this method requires multiple calls and cannot be done in parallel, leading to slower execution. The refine method also allows for iterative building upon previous documents, but it uses a different approach and is depicted in Fig. 7. The map-reduce method is a single call to the language model for each document, making it faster in execution. However, it may not produce as comprehensive of an answer as the stuff or refine methods. The map-rerank method also involves a single call to the language model for each document, but it selects the document with the highest score, potentially producing a more accurate answer. However, it may not be as comprehensive as the other methods. In summary, the trade-offs between these methods involve speed of execution versus comprehensiveness and accuracy of the answers.


### Question 3

**How does LangChain support the use of agents and tools for dynamic task execution?**

**Answer:**

 LangChain supports the use of agents and tools for dynamic task execution by using a language model as the planner and an action agent as the executor. This allows for a continuous flow of actions and inputs, where outputs from previous steps are utilized as inputs for subsequent ones. The framework also incorporates the use of tools, which are interfaces that facilitate interactions between agents and the external world, allowing for purposeful actions to be performed. This enables the implementation of various use cases, such as autonomous agents, chatbots, code understanding agents, extraction, question answering, summarization, and analyzing structured data. Additionally, LangChain utilizes memory implementation to store and retrieve information from previous conversations, allowing for a more personalized and contextualized experience for the user. 


In [ ]:

display(Markdown("### ⚠️ Limitation Check"))
hard_question = "What are the ethical implications of using LangChain in military AI applications?"
display(Markdown(f"**Question:** {hard_question}"))

hard_answer = qa_chain.run(hard_question)
display(Markdown(f"**Answer (may be incomplete or hallucinated):**\n\n{hard_answer}"))

### ⚠️ Limitation Check

**Question:** What are the ethical implications of using LangChain in military AI applications?

**Answer (may be incomplete or hallucinated):**

 I don't know. The context provided does not mention anything about military AI applications or any potential ethical implications.

In [ ]:
"""
User question
   ↓
Retriever finds matching chunks from FAISS
   ↓
Chunks + question fed to OpenAI model
   ↓
Answer is generated using only the retrieved context

"""

In [ ]:
# --- PART 2: Zero-Shot ReAct Agent with Tools ---
!pip install -q jupyter langchain-experimental openai wikipedia

In [ ]:
# --- PART 2: Zero-Shot ReAct Agent with Tools ---

from langchain.agents import Tool, initialize_agent, AgentType, load_tools
from langchain.llms import OpenAI
from langchain_experimental.tools.python.tool import PythonREPLTool

# Set up LLM
llm = OpenAI(temperature=0)

In [ ]:
""" Keep in mind that LangChain is getting updates on a continiously basis

ImportError                               Traceback (most recent call last)
<ipython-input-24-677dc4aad66a> in <cell line: 0>()
      4 from langchain.utilities import WikipediaAPIWrapper
      5 from langchain_experimental.tools.python.tool import PythonREPLTool
----> 6 from langchain.agents.agent_toolkits import LLMMathTool
      7
      8 # Load LLM (same as used in Part 1)

ImportError: cannot import name 'LLMMathTool' from 'langchain.agents.agent_toolkits' (/usr/local/lib/python3.11/dist-packages/langchain/agents/agent_toolkits/__init__.py)


Reason:  Thanks —This is due to another recent LangChain update. The import path for LLMMathTool has changed.


"""

In [ ]:
# Load Wikipedia + Math tools using load_tools
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Add Python REPL manually (from experimental package)
tools.append(PythonREPLTool())

In [ ]:
""" model used by default: model_name = "text-davinci-003"

In [ ]:
from IPython.display import display, Markdown

# Initialize Zero-Shot ReAct Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

<ipython-input-29-0eb67f0abe00>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
# DEMO 1 — Wikipedia + LLM Math
question1 = "What is the population of France, and what is the square root of that number?"
display(Markdown("### 🤖 Agent Tool Use Example — Reason + Wikipedia + Math"))
display(Markdown(f"**Question:** {question1}"))
answer1 = agent.run(question1)
display(Markdown(f"**Answer:**\n\n{answer1}"))

### 🤖 Agent Tool Use Example — Reason + Wikipedia + Math

**Question:** What is the population of France, and what is the square root of that number?



> Entering new AgentExecutor chain...
 I should use Wikipedia to find the population of France and then use the Calculator to find the square root.
Action: wikipedia
Action Input: France population
Observation: Page: Demographics of France
Summary: The demography of France is monitored by the Institut national d'études démographiques (INED) and the Institut national de la statistique et des études économiques (INSEE). As of 1 January 2025, 66,352,000 people lived in Metropolitan France, while 2,254,000 lived in overseas France, for a total of 68,606,000 inhabitants in the French Republic. In January 2022, the population of France officially reached the 68,000,000 mark. The population of France is growing by 1,000,000 people every three years- an average annual increase of 340,000 people, or +0.6%.
France was historically Europe's most populous country. During the Middle Ages, more than one-quarter of Europe's total population was French; by the seventeenth century, this had decreased

**Answer:**

The population of France is 66,352,000 and the square root of that number is approximately 8145.67. The population of Canada is 41,500,000 and the square root of that number is approximately 6442.05.

In [ ]:
#### One Solution:

##  question = "Only tell me the current population of France and its square root. Do not include Canada or other countries."

In [ ]:
# DEMO 2 — Python REPL
question2 = "What is the cosine of 60 degrees plus the natural log of 10?"
display(Markdown("### 🧮 Agent Tool Use Example — Python REPL"))
display(Markdown(f"**Question:** {question2}"))
answer2 = agent.run(question2)
display(Markdown(f"**Answer:**\n\n{answer2}"))

### 🧮 Agent Tool Use Example — Python REPL

**Question:** What is the cosine of 60 degrees plus the natural log of 10?



> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: cos(60) + log(10)
Observation: Answer: 1.3501721125788895
Thought:

 I should round the answer to two decimal places.
Action: Python_REPL
Action Input: round(1.3501721125788895, 2)
Observation: 
Thought: I now know the final answer.
Final Answer: 1.35

> Finished chain.


**Answer:**

1.35